## Demo for Random TODOList

- 我的 Random 应该加在 GitHub Action 上面

## Refer

- https://pypi.org/project/py-trello/
- https://networkx.org/documentation/stable/tutorial.html

In [1]:
import configparser
import pandas as pd
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

## Connect to Trello

In [2]:
from trello import TrelloClient

trello_client = TrelloClient(
    api_key=config["defaut"]["api_key"],
    api_secret=config["defaut"]["api_secret"],
    token=config["defaut"]["token"]
)
all_boards = trello_client.list_boards()
last_board = all_boards[0]
print(last_board.name)
print(all_boards[0].list_lists()[3].id)

Hi-Kanban
631058bee13e0d048d72c450


In [3]:
import re
category_str = "Knowledge|Output|ACG|Keep|Book"

def map_to(x):
    weights = re.findall('\((.*?)\)[ ]*\|', x.name)
    weight = int(weights[0]) if len(weights) >= 1 else 5
    categorys = re.findall("^(" + category_str + ")[^(^|.]*", x.name)
    category = categorys[0] if len(categorys) >= 1 else "null"
    name = re.sub("^" + category + "[^|.]*\|", "", x.name).strip()
    return { 
        "name": name,
        "weight": weight, 
        "category": category, 
        "labels": [ label.name for label in x.labels ],
        "id": x.id,
        "title": x.name, 
        "short_url": x.shortUrl
    }

In [4]:
history = pd.read_csv('table.csv')
history

,Date,Title,Category
0,2023-06,VA-11 Hall-A: Cyberpunk Bartender Action,Knowledge
1,2023-07,Apple Game Porting Toolkit Research,Knowledge
2,2023-08,The Legend of Zelda: Breath of the Wild - nira...,ACG


In [5]:
todo_list = last_board.get_list("631058bee13e0d048d72c450")
l = pd.DataFrame(list(map(map_to, todo_list.list_cards())))
l = l.set_index("id")
previous_category = history.iloc[-1]['Category']
if previous_category == "ACG":
    l.loc[l['category'] == 'ACG', 'weight'] /= 2 # if last month is ACG
print(len(l))
print(l.groupby(['category'])['weight'].sum() / l['weight'].sum() * 100)
l[-10:]

7
category
ACG           7.692308
Book          7.692308
Keep         15.384615
Knowledge     7.692308
Output       53.846154
null          7.692308
Name: weight, dtype: float64


,name,weight,category,labels,title,short_url
id,,,,,,
648fcca93ff71a2867b44ad6,LLaMA 《Prompt Engineering》《Building System》《La...,30,Output,"[Learn, Reading Notes]",Output(30) | LLaMA 《Prompt Engineering》《Buildi...,https://trello.com/c/0Y0q6PVo
62dd197bc71a2589a9aee745,《羽毛球基础精要》 | Improving Badminton Skills | 提升羽毛球技术,10,Keep,[Sport],Keep(10) | 《羽毛球基础精要》 | Improving Badminton Ski...,https://trello.com/c/cJ7MZSPe
63903879fe363100e5ceb378,Persona 5 Royal | 女神异闻录五 皇家版,5,ACG,[game],ACG(10) | Persona 5 Royal | 女神异闻录五 皇家版,https://trello.com/c/RjWy6yJp
64d047dd6f82c97369e7cad0,CyberSecurity | Add two-factor authentication ...,5,null,[Learn],CyberSecurity | Add two-factor authentication ...,https://trello.com/c/ZvWSPTvc
6472fe7883627f70d789cd1f,The Mythical Man-Month | Identify the challeng...,5,Book,"[Learn, Reading Notes]",Book | The Mythical Man-Month | Identify the c...,https://trello.com/c/XOm4sDK1
62b57d8d780dde3647e1ba2d,It is necessary to take photos to document lif...,5,Output,[Life Habit],Output | It is necessary to take photos to doc...,https://trello.com/c/Hf8Ujtkp
60b9d1aa383fca278c6ca85a,阮一峰《TypeScript教程》| Review the TypeScript code ...,5,Knowledge,[Learn],Knowledge | 阮一峰《TypeScript教程》| Review the Type...,https://trello.com/c/CdY3SLTg


## Connect to telegram

In [6]:
from telethon import TelegramClient, events, sync
import socks
import os

SESSION = os.environ.get('TG_SESSION', 'interactive')
API_ID = int(config["telegram"]["tg_api_id"])
API_HASH = config["telegram"]["tg_api_hash"]
PROXY = (socks.SOCKS5, '192.168.10.119', 1080)

if f"{SESSION}.session" in os.listdir():
    os.remove(f"{SESSION}.session")

import sys
import time
client = TelegramClient(SESSION, API_ID, API_HASH, proxy=PROXY)
from IPython.display import clear_output
await client.start()

clear_output()
channel_username = 'niracler_channel'  # your channel

In [7]:
# data = [
#     ['2023-07', '最近做的工作跟系统底层打交道的情况实在是不少，需要补一下一些虚拟化方面的知识', 'Knowledge'],
# ]

# df = pd.DataFrame(data, columns=['Date', 'Title', 'Category'])
# df.to_csv('table.csv', index=False)

## Random Now ~~

In [8]:
import random, datetime, time

user_input = input("确定要继续运行吗？(Y/N): ")    

def rs():
    return random.choices(
        population=list(l.index),
        weights=list(l.weight),
        k=1
    )

e = datetime.datetime.now()
month = datetime.datetime.today().month

for i in range(10):
    time.sleep(3) # Delay for 3 seconds.
    random_id = rs()[0]
    if i < 9:
        print("不是: " + str(l.loc[random_id]['name']))
        
msg = "{month}月份就是这个了(id:{id}):\n\n{title} | {labels}\n\n恭喜恭喜!!!({time})\nurl: {url}\n#random_todolist".format(
    month=month+1,
    id=random_id,
    url=l.loc[random_id]['short_url'],
    title=l.loc[random_id]['title'],
    labels=l.loc[random_id]['labels'], 
    time=e.strftime("%Y-%m-%d %H:%M:%S")
)

print("\n" + msg)

## Sync to ...
        
for card in todo_list.list_cards():
    if(card.id == random_id):
        card.change_list("6024cd248ecb43309b5eb2d0")
        
message = await client.send_message(entity=channel_username, message=msg)
await client.pin_message(channel_username, message, notify=True)

from datetime import datetime
from dateutil.relativedelta import relativedelta

if len(history) == 0:
    current_date = datetime.now().strftime('%Y-%m')
else:
    previous_date = history.iloc[-1]['Date']
    previous_date = datetime.strptime(previous_date, '%Y-%m')
    current_date = previous_date + relativedelta(months=1)
    current_date = current_date.strftime('%Y-%m')

new_row = {'Date': current_date, 'Title': l.loc[random_id]['name'], 'Category': l.loc[random_id]['category'].replace("|", "-")}
new_df = pd.DataFrame(new_row, index=[0])

history = pd.concat([history, new_df], ignore_index=True)
markdown_table = history.to_markdown(index=False)
history

# 更新 README 文档
readme_path = 'README.md'

with open(readme_path, 'r') as file:
    readme_content = file.read()

# 找到要替换的标记，例如 <!-- TABLE_START --> 和 <!-- TABLE_END -->
table_start = '<!-- TABLE_START -->\n'
table_end = '\n<!-- TABLE_END -->'

# 替换标记位置的内容为 Markdown 表格
regex = f"{re.escape(table_start)}(.*?)\n{re.escape(table_end)}"
updated_readme_content = re.sub(regex, f"{table_start}\n{markdown_table}\n{table_end}", readme_content, flags=re.DOTALL)
updated_readme_content

# 写入更新后的 README 文件
with open(readme_path, 'w') as file:
    file.write(updated_readme_content)

print("README 文件已更新！")
history.to_csv('table.csv', index=False)
history

确定要继续运行吗？(Y/N):  


不是: The Witcher 3 | niracler
不是: The Witcher 3 | niracler
不是: LLaMA 《Prompt Engineering》《Building System》和《LangChain》 | niracler
不是: 画画技能培训 | niracler
不是: Animal Crossing: New Horizons | niracler
不是: LLaMA 《Prompt Engineering》《Building System》和《LangChain》 | niracler
不是: 《深入浅出nodejs》| niracler
不是: LLaMA 《Prompt Engineering》《Building System》和《LangChain》 | niracler
不是: Danganronpa V3: Killing Harmony | niracler

8月份就是这个了(id:62b576a357dd3e4a8c5de383):

ACG(30) | The Legend of Zelda: Breath of the Wild | niracler | []

恭喜恭喜!!!(2023-07-25 03:33:47)
url: https://trello.com/c/FjQNWFZY
#random_todolist
README 文件已更新！


,Date,Title,Category
0,2023-07,最近做的工作跟系统底层打交道的情况实在是不少，需要补一下一些虚拟化方面的知识,Knowledge
1,2023-08,The Legend of Zelda: Breath of the Wild | nira...,ACG


In [9]:
print("hello world")

hello world
